# Topic Visualization with pyLDAvis

In [1]:
import pandas as pd
import random

import string 
import re

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis.sklearn


In [2]:
pyLDAvis.enable_notebook()

other_punct = ['’', '“','”', "''", '""', '', '..', '...', '``', '—', '->', '’ ’']
eng_stopwords_and_punct = stopwords.words('english') + list(string.punctuation) + other_punct

# TweetTokenizer will put all text in the tweet to lowercase, strip out usernames/handles and reduce reated chars in words
tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True) 

lemmatizer = WordNetLemmatizer()

# Functions
def split_content_by_sentiment(region_df):
    '''
    Create Series for each sentiment class from the provided dataframe
    '''
    pos = region_df[region_df['sentiment'] == 'positive']['content']
    neg = region_df[region_df['sentiment'] == 'negative']['content']
    neu = region_df[region_df['sentiment'] == 'neutral']['content']
    return [pos, neg, neu]

def clean_tokenize_lemmatize_tweet(tweet):
    ''' 
    Apply text cleaning, tokenization then lemmatization on the given tweet 
    '''
    #remove urls
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
    #remove RT 
    tweet = re.sub(r'^RT\s+', '', tweet)
    #remove the # symbol
    tweet = re.sub('#', '', tweet)
    #remove words containing numbers
    tweet = re.sub(r'\w*\d\w*', '', tweet)
    #tokenize
    tokens = tweet_tokenizer.tokenize(tweet)
    # remove stopwords and punctuation
    tokens = [token for token in tokens if token not in eng_stopwords_and_punct]
    # remove tokens that are only 1 char in length
    tokens = [token for token in tokens if len(token)>1]
    #lemmatize
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens


def perform_topic_modeling(content, topic_num, top_word_count):
    '''
    Perform topic modeling using on the provided corpus for the 
    designated number of topics and top word count
    '''
    cv = CountVectorizer(max_df=0.95, 
                         min_df=0.01, 
                         strip_accents='unicode', 
                         tokenizer=clean_tokenize_lemmatize_tweet, 
                         ngram_range=(1,2))

    lda = LatentDirichletAllocation(max_iter=10, 
                                    learning_method='online', 
                                    n_components=topic_num, 
                                    random_state=42)

    # Unsupervised learning...no need to do a test/train split here ;)
    dtm = cv.fit_transform(content)

    # This might take awhile if you are dealing with a large amount of documents!
    lda.fit(dtm)

    return (lda, dtm, cv)


In [3]:
# Read in all US Tweets and create Series for entire US and Regions of interest 
vetted_usa_tweets = pd.read_csv('../data/vetted_usa_tweets.csv')

west = vetted_usa_tweets[vetted_usa_tweets['region'] == 'West']
midwest = vetted_usa_tweets[vetted_usa_tweets['region'] == 'Midwest']
southwest = vetted_usa_tweets[vetted_usa_tweets['region'] == 'Southwest']
southeast = vetted_usa_tweets[vetted_usa_tweets['region'] == 'Southeast']
northeast = vetted_usa_tweets[vetted_usa_tweets['region'] == 'Northeast']

# Series of Tweets by Region
us_tweets = vetted_usa_tweets['content']
west_tweets = west['content']
midwest_tweets = midwest['content']
southwest_tweets = southwest['content']
southeast_tweets = southeast['content']
northeast_tweets = northeast['content']

# Series of Tweets by Region and Sentiment
us_tweets_by_sentiment = split_content_by_sentiment(vetted_usa_tweets)
west_tweets_by_sentiment = split_content_by_sentiment(west)
midwest_tweets_by_sentiment = split_content_by_sentiment(midwest)
southwest_tweets_by_sentiment = split_content_by_sentiment(southwest)
southeast_tweets_by_sentiment = split_content_by_sentiment(southeast)
northeast_tweets_by_sentiment = split_content_by_sentiment(northeast)


## US Topics

In [4]:
# model topics in all the Tweets
usa_tm_tuple = perform_topic_modeling(us_tweets, 8, 30)
pyLDAvis.sklearn.prepare(usa_tm_tuple[0], usa_tm_tuple[1], usa_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.180666 -0.062312       1        1  20.494013
1      0.229455 -0.174328       2        1  13.001470
4     -0.036142  0.189399       3        1  12.917189
6      0.029853  0.155566       4        1  12.692270
3     -0.133170  0.138530       5        1  11.587619
7      0.216696 -0.098278       6        1  11.535499
0     -0.355552 -0.329037       7        1   9.928299
2     -0.131806  0.180460       8        1   7.843641, topic_info=           Term          Freq         Total Category  logprob  loglift
44        class   7225.000000   7225.000000  Default  30.0000  30.0000
55       course   5503.000000   5503.000000  Default  29.0000  29.0000
196      online  19915.000000  19915.000000  Default  28.0000  28.0000
215         pay   3724.000000   3724.000000  Default  27.0000  27.0000
293    teaching   6357.000000   6357.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
185     meeting    239.786046    428.124467   Topic8  -4.6811   1.9658
40        check    535.157731   1282.287084   Topic8  -3.8783   1.6716
232  psychology    274.957634    561.209620   Topic8  -4.5442   1.8320
196      online    743.298907  19915.833452   Topic8  -3.5497  -0.7427
114         get    397.361889   2348.910646   Topic8  -4.1760   0.7686

[342 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
0         1  0.997019              able
3         3  0.998360        affordable
4         7  0.998987           algebra
5         7  0.998798  algebra calculus
7         4  0.999493              also
...     ...       ...               ...
335       2  0.998896             would
336       6  0.999557             write
337       1  0.997311           writing
338       6  0.999547              year
339       8  0.999337              zoom

[387 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 5, 7, 4, 8, 1, 3])

### US Positive Tweets

In [5]:
# model topics in Positive Tweets
usa_pos_tm_tuple = perform_topic_modeling(us_tweets_by_sentiment[0], 5, 30)
pyLDAvis.sklearn.prepare(usa_pos_tm_tuple[0], usa_pos_tm_tuple[1], usa_pos_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.105515  0.028626       1        1  28.791943
3      0.091477 -0.316008       2        1  18.970539
1      0.024689  0.277368       3        1  18.934096
2     -0.415255 -0.028088       4        1  17.429165
4      0.193574  0.038102       5        1  15.874257, topic_info=         Term          Freq         Total Category  logprob  loglift
204    online  14298.000000  14298.000000  Default  30.0000  30.0000
305  teaching   4650.000000   4650.000000  Default  29.0000  29.0000
60     course   4111.000000   4111.000000  Default  28.0000  28.0000
162  learning   5978.000000   5978.000000  Default  27.0000  27.0000
304   teacher   2741.000000   2741.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
118       get    744.588479   1834.955886   Topic5  -3.9992   0.9385
46      check    552.109656   1012.774416   Topic5  -4.2983   1.2338
125     grade    452.422029    675.060785   Topic5  -4.4974   1.4403
333   virtual    630.089597   1676.726952   Topic5  -4.1662   0.8617
197       new    547.736616   2070.878926   Topic5  -4.3063   0.5105

[254 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         2  0.998995     able
1         3  0.034533  academy
1         5  0.963483  academy
5         4  0.999207  algebra
14        5  0.997492      art
...     ...       ...      ...
351       3  0.995461  writing
352       1  0.721210     year
352       2  0.264061     year
352       5  0.014612     year
354       3  0.997127     zoom

[314 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3, 5])

### US Negative Tweets

In [6]:
# model topics in Negative Tweets
usa_neg_tm_tuple = perform_topic_modeling(us_tweets_by_sentiment[1], 3, 30)
pyLDAvis.sklearn.prepare(usa_neg_tm_tuple[0], usa_neg_tm_tuple[1], usa_neg_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.351851  0.080964       1        1  50.456029
2      0.091570 -0.293763       2        1  43.091554
0      0.260281  0.212798       3        1   6.452417, topic_info=                  Term         Freq        Total Category  logprob  loglift
284                pay  1962.000000  1962.000000  Default  30.0000  30.0000
212           learning   744.000000   744.000000  Default  29.0000  29.0000
354             school   872.000000   872.000000  Default  28.0000  28.0000
388            student   556.000000   556.000000  Default  27.0000  27.0000
115              essay   932.000000   932.000000  Default  26.0000  26.0000
..                 ...          ...          ...      ...      ...      ...
106          education    83.496324   390.811255   Topic3  -3.7737   1.1973
388            student    87.705769   556.329495   Topic3  -3.7245   0.8933
88   distance learning    81.704529   375.601337   Topic3  -3.7954   1.2153
87            distance    81.162568   459.453218   Topic3  -3.8021   1.0071
212           learning    84.254468   744.373410   Topic3  -3.7647   0.5620

[150 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
6         3  0.994141  afterschoolworks
7         1  0.998252           algebra
7         3  0.003110           algebra
8         1  0.998274  algebra calculus
8         3  0.004303  algebra calculus
...     ...       ...               ...
442       3  0.994141  would disastrous
443       1  0.996852             write
443       3  0.002455             write
449       2  0.997609              year
449       3  0.004843              year

[206 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

### US Neutral Tweets

In [7]:
# model topics in Neutral Tweets
usa_neu_tm_tuple = perform_topic_modeling(us_tweets_by_sentiment[2], 5, 30)
pyLDAvis.sklearn.prepare(usa_neu_tm_tuple[0], usa_neu_tm_tuple[1], usa_neu_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.023016 -0.207048       1        1  28.843739
0     -0.179592 -0.025738       2        1  26.260757
1     -0.269344  0.177477       3        1  17.068059
4      0.185069 -0.221456       4        1  14.933296
2      0.286883  0.276766       5        1  12.894149, topic_info=                 Term         Freq        Total Category  logprob  loglift
59               exam   703.000000   703.000000  Default  30.0000  30.0000
125            online  2572.000000  2572.000000  Default  29.0000  29.0000
24              class   861.000000   861.000000  Default  28.0000  28.0000
164            school   855.000000   855.000000  Default  27.0000  27.0000
32             course   725.000000   725.000000  Default  26.0000  26.0000
..                ...          ...          ...      ...      ...      ...
222              zoom    19.436404    65.706666   Topic5  -5.4702   0.8303
44   distancelearning    91.439583   420.462011   Topic5  -3.9216   0.5227
27          classroom    38.946427   192.311803   Topic5  -4.7751   0.4515
203               via    58.043106   296.841348   Topic5  -4.3761   0.4164
186          teaching    79.407129   577.285346   Topic5  -4.0627   0.0646

[221 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         4  0.989235      access
1         1  0.987999        also
2         2  0.912856  assignment
2         4  0.078889  assignment
3         1  0.992080         aug
...     ...       ...         ...
219       4  0.559168        work
221       1  0.997913        year
222       1  0.532670        zoom
222       2  0.167411        zoom
222       5  0.289164        zoom

[248 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 5, 3])

## West Region Topics

In [8]:
# model topics in all West Region Tweets
w_tm_tuple = perform_topic_modeling(west_tweets, 5, 30)
pyLDAvis.sklearn.prepare(w_tm_tuple[0], w_tm_tuple[1], w_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.122829  0.134395       1        1  26.363943
4     -0.004304  0.118308       2        1  25.812551
2      0.188783 -0.324123       3        1  24.465980
0      0.246623  0.180615       4        1  14.703798
1     -0.308273 -0.109195       5        1   8.653728, topic_info=                  Term         Freq        Total Category  logprob  loglift
178             online  3262.000000  3262.000000  Default  30.0000  30.0000
223             school  1684.000000  1684.000000  Default  29.0000  29.0000
40               class  1214.000000  1214.000000  Default  28.0000  28.0000
142           learning  2305.000000  2305.000000  Default  27.0000  27.0000
261            teacher  1017.000000  1017.000000  Default  26.0000  26.0000
..                 ...          ...          ...      ...      ...      ...
101                get   194.638606   695.919769   Topic5  -3.3912   1.1731
250            student   142.487024  1237.711068   Topic5  -3.7031   0.2854
142           learning   128.329740  2305.475752   Topic5  -3.8077  -0.4413
61            distance    89.233246  1415.794611   Topic5  -4.1711  -0.3170
62   distance learning    78.723363  1273.780283   Topic5  -4.2964  -0.3366

[257 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         2  0.991080        able
1         4  0.995600     academy
2         1  0.995929      access
5         2  0.989527     another
9         3  0.994992  assignment
...     ...       ...         ...
308       2  0.988864       world
309       1  0.002897       would
309       5  0.996531       would
311       1  0.242194        year
311       2  0.756203        year

[340 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 3, 1, 2])

In [9]:
# model topics in Positive West Region Tweets
w_pos_tm_tuple = perform_topic_modeling(west_tweets_by_sentiment[0], 3, 30)
pyLDAvis.sklearn.prepare(w_pos_tm_tuple[0], w_pos_tm_tuple[1], w_pos_tm_tuple[2])


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.152614  0.185222       1        1  43.575010
0     -0.113486 -0.204314       2        1  35.371109
2      0.266101  0.019093       3        1  21.053881, topic_info=                  Term         Freq        Total Category  logprob  loglift
147           learning  1936.000000  1936.000000  Default  30.0000  30.0000
185             online  2623.000000  2623.000000  Default  29.0000  29.0000
63            distance  1157.000000  1157.000000  Default  28.0000  28.0000
64   distance learning  1037.000000  1037.000000  Default  27.0000  27.0000
51              course   799.000000   799.000000  Default  26.0000  26.0000
..                 ...          ...          ...      ...      ...      ...
273            teacher   155.794469   662.130687   Topic3  -4.3063   0.1112
43           classroom   126.573640   259.701713   Topic3  -4.5140   0.8394
184                one   125.328397   249.236550   Topic3  -4.5239   0.8706
101        free online   115.687991   176.041555   Topic3  -4.6040   1.1383
176               need   121.060876   436.671727   Topic3  -4.5586   0.2752

[203 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         3  0.996165     academy
2         1  0.995254      access
3         1  0.009714        also
3         3  0.990861        also
11        2  0.992221  assignment
...     ...       ...         ...
315       2  0.008064     working
316       2  0.985148    workshop
319       1  0.964788        year
319       2  0.027678        year
319       3  0.007908        year

[295 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [10]:
# model topics in Negative West Region Tweets
w_neg_tm_tuple = perform_topic_modeling(west_tweets_by_sentiment[1], 3, 30)
pyLDAvis.sklearn.prepare(w_neg_tm_tuple[0], w_neg_tm_tuple[1], w_neg_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.033957 -0.239492       1        1  48.280733
1     -0.298283  0.098063       2        1  28.776200
0      0.264325  0.141428       3        1  22.943067, topic_info=             Term        Freq       Total Category  logprob  loglift
230           pay  176.000000  176.000000  Default  30.0000  30.0000
280        school  202.000000  202.000000  Default  29.0000  29.0000
37          class  133.000000  133.000000  Default  28.0000  28.0000
212        online  232.000000  232.000000  Default  27.0000  27.0000
213  online class   60.000000   60.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
345             u   27.846509   30.503704   Topic3  -4.2328   1.3810
37          class   98.184806  133.738606   Topic3  -2.9727   1.1631
212        online   99.901000  232.002505   Topic3  -2.9554   0.6296
285      semester   19.494554   51.992803   Topic3  -4.5894   0.4912
48         course   15.517041   53.924382   Topic3  -4.8176   0.2265

[160 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
5         2  0.984581  afterschoolworks
9         1  0.935134              also
14        3  0.982985        assignment
21        3  0.925890           biology
24        3  0.939987          business
...     ...       ...               ...
368       2  0.598483             would
369       2  0.984697  would disastrous
370       3  0.979488             write
375       1  0.975558              year
375       3  0.032519              year

[185 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [11]:
# model topics in Neutral West Region Tweets
w_neu_tm_tuple = perform_topic_modeling(west_tweets_by_sentiment[2], 3, 30)
pyLDAvis.sklearn.prepare(w_neu_tm_tuple[0], w_neu_tm_tuple[1], w_neu_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.068849 -0.289496       1        1  54.906195
2     -0.270756  0.193731       2        1  25.376489
0      0.339606  0.095765       3        1  19.717316, topic_info=             Term        Freq       Total Category  logprob  loglift
49      elearning  147.000000  147.000000  Default  30.0000  30.0000
193     voiceover  125.000000  125.000000  Default  29.0000  29.0000
64         fiverr  122.000000  122.000000  Default  28.0000  28.0000
68            get   99.000000   99.000000  Default  27.0000  27.0000
114  online class  112.000000  112.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
107          need   46.106857   78.297120   Topic3  -3.5144   1.0941
14          check   26.803079   53.945050   Topic3  -4.0568   0.9242
140        record   11.937762   12.920422   Topic3  -4.8657   1.5446
157       service   12.358165   15.678006   Topic3  -4.8310   1.3857
187         video   12.638833   42.275911   Topic3  -4.8086   0.4162

[135 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2         2  0.953869  assignment
4         1  0.967854      august
5         1  0.970955        back
12        3  0.983965    business
14        1  0.481972       check
...     ...       ...         ...
206       2  0.954799        work
207       1  0.286519    workshop
207       2  0.687646    workshop
209       2  0.929257       write
210       1  0.990585        year

[141 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

## Southwest Region Topics

In [21]:
# model topics in all Southwest Region Tweets
sw_tm_tuple = perform_topic_modeling(southwest_tweets, 5, 30)
pyLDAvis.sklearn.prepare(sw_tm_tuple[0], sw_tm_tuple[1], sw_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.032376 -0.087501       1        1  23.392495
2      0.230015  0.158954       2        1  21.097297
0      0.101158  0.038990       3        1  20.530659
3     -0.027599 -0.242503       4        1  19.149568
4     -0.335950  0.132061       5        1  15.829981, topic_info=                 Term         Freq        Total Category  logprob  loglift
219            school   653.000000   653.000000  Default  30.0000  30.0000
172            online  1854.000000  1854.000000  Default  29.0000  29.0000
222          semester   380.000000   380.000000  Default  28.0000  28.0000
253          teaching   583.000000   583.000000  Default  27.0000  27.0000
302              year   290.000000   290.000000  Default  26.0000  26.0000
..                ...          ...          ...      ...      ...      ...
294              week    79.965801   165.137348   Topic5  -4.0779   1.1181
104             great    82.355985   178.873164   Topic5  -4.0485   1.0676
135          learning   144.773634   845.074273   Topic5  -3.4843   0.0790
242           student   115.422390   631.700056   Topic5  -3.7109   0.1434
59   distancelearning    72.062687   243.707465   Topic5  -4.1820   0.6248

[266 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         5  0.983444        able
2         4  0.976130     account
3         4  0.990972    activity
4         1  0.984850  affordable
5         2  0.981507     algebra
...     ...       ...         ...
300       2  0.988904       write
301       2  0.990059     writing
302       1  0.006896        year
302       4  0.003448        year
302       5  0.989528        year

[383 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 4, 5])

### Southwest Positive Tweets

In [12]:
# model topics in Positive Southwest Region Tweets
sw_pos_tm_tuple = perform_topic_modeling(southwest_tweets_by_sentiment[0], 3, 30)
pyLDAvis.sklearn.prepare(sw_pos_tm_tuple[0], sw_pos_tm_tuple[1], sw_pos_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.140048  0.164675       1        1  46.644948
0     -0.165422 -0.155801       2        1  38.631137
2      0.305470 -0.008873       3        1  14.723914, topic_info=                 Term        Freq       Total Category  logprob  loglift
296          teaching  461.000000  461.000000  Default  30.0000  30.0000
330   virtuallearning  244.000000  244.000000  Default  29.0000  29.0000
328           virtual  218.000000  218.000000  Default  28.0000  28.0000
66           distance  266.000000  266.000000  Default  27.0000  27.0000
119              help  269.000000  269.000000  Default  26.0000  26.0000
..                ...         ...         ...      ...      ...      ...
43              class   57.964404  498.862852   Topic3  -4.1582  -0.2368
290              take   46.378380  138.174360   Topic3  -4.3812   0.8240
294           teacher   47.774925  312.906494   Topic3  -4.3516   0.0363
283           student   45.818367  510.656629   Topic3  -4.3934  -0.4953
68   distancelearning   40.804322  189.042316   Topic3  -4.5093   0.3825

[197 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         3  0.992010        able
3         1  0.974384  affordable
4         3  0.973639     algebra
9         2  0.207540     amazing
9         3  0.773559     amazing
...     ...       ...         ...
352       1  0.436951        year
352       2  0.561182        year
352       3  0.004284        year
354       2  0.020030        zoom
354       3  0.961450        zoom

[268 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

### Southwest Negative Tweets

In [13]:
# model topics in Negative Southwest Region Tweets
sw_neg_tm_tuple = perform_topic_modeling(southwest_tweets_by_sentiment[1], 3, 30)
pyLDAvis.sklearn.prepare(sw_neg_tm_tuple[0], sw_neg_tm_tuple[1], sw_neg_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.088463  0.128348       1        1  67.384147
0     -0.103297 -0.121895       2        1  16.687099
2      0.191760 -0.006453       3        1  15.928754, topic_info=             Term       Freq       Total Category  logprob  loglift
345           pay  55.000000   55.000000  Default  30.0000  30.0000
255      learning  64.000000   64.000000  Default  29.0000  29.0000
64          class  72.000000   72.000000  Default  28.0000  28.0000
410        school  76.000000   76.000000  Default  27.0000  27.0000
481      teaching  94.000000   94.000000  Default  26.0000  26.0000
..            ...        ...         ...      ...      ...      ...
186      get sick   3.894279    4.856798   Topic3  -5.1703   1.6162
324  online class  12.042452   27.049706   Topic3  -4.0414   1.0278
64          class  19.217975   72.310085   Topic3  -3.5739   0.5119
323        online  10.685918  202.325259   Topic3  -4.1609  -1.1039
504             u   5.171838   21.302498   Topic3  -4.8866   0.4214

[174 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
5         2  0.951381             added
6         2  0.952813         added sex
9         3  0.948368  algebra calculus
14        2  0.910830            answer
16        2  0.953304            appeal
...     ...       ...               ...
533       3  0.044315              work
540       1  0.950525             would
542       1  0.210372             write
542       3  0.736302             write
546       1  0.975888              year

[210 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

### Southewst Neutral Tweets

In [14]:
# model topics in Neutral Southwest Region Tweets
sw_neu_tm_tuple = perform_topic_modeling(southwest_tweets_by_sentiment[2], 3, 30)
pyLDAvis.sklearn.prepare(sw_neu_tm_tuple[0], sw_neu_tm_tuple[1], sw_neu_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.134548  0.153892       1        1  43.485817
0      0.286219 -0.006018       2        1  39.207224
1     -0.151671 -0.147875       3        1  17.306960, topic_info=                 Term        Freq       Total Category  logprob  loglift
213            school   80.000000   80.000000  Default  30.0000  30.0000
280   virtuallearning   76.000000   76.000000  Default  29.0000  29.0000
133          learning   98.000000   98.000000  Default  28.0000  28.0000
63   distancelearning   42.000000   42.000000  Default  27.0000  27.0000
157            online  209.000000  209.000000  Default  26.0000  26.0000
..                ...         ...         ...      ...      ...      ...
240           student   12.485890   53.349610   Topic3  -4.0625   0.3018
28              check   10.170421   34.275249   Topic3  -4.2676   0.5391
269               use    9.204964   24.351299   Topic3  -4.3674   0.7812
153              need    8.268665   19.893064   Topic3  -4.4746   0.8762
72             edtech    7.876700   34.542222   Topic3  -4.5232   0.2758

[160 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
2         2  0.959728          account
3         2  0.959774  account website
10        3  0.832860              app
14        1  0.171012             back
14        3  0.812308             back
...     ...       ...              ...
288       1  0.903733             week
294       1  0.444692             year
294       3  0.533631             year
295       1  0.173641    year distance
295       3  0.868204    year distance

[208 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

## Midwest Region Topics

In [15]:
# model topics in all Midwest Region Tweets
mw_tm_tuple = perform_topic_modeling(midwest_tweets, 5, 30)
pyLDAvis.sklearn.prepare(mw_tm_tuple[0], mw_tm_tuple[1], mw_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.006820 -0.060956       1        1  32.238272
1     -0.069664  0.159591       2        1  25.993144
0     -0.288844 -0.146821       3        1  19.360068
3      0.089290  0.241006       4        1  11.335064
4      0.276038 -0.192820       5        1  11.073453, topic_info=         Term         Freq        Total Category  logprob  loglift
33      class   879.000000   879.000000  Default  30.0000  30.0000
167    online  2445.000000  2445.000000  Default  29.0000  29.0000
43     course   758.000000   758.000000  Default  28.0000  28.0000
222  semester   553.000000   553.000000  Default  27.0000  27.0000
251  teaching   752.000000   752.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
264     today    76.005114   226.092804   Topic5  -4.1042   1.1105
132      like    78.774786   319.562685   Topic5  -4.0684   0.8003
166       one    68.657694   236.158196   Topic5  -4.2059   0.9653
167    online    96.657687  2445.640759   Topic5  -3.8638  -1.0303
92       good    61.284171   151.828825   Topic5  -4.3195   1.2934

[259 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         4  0.979213        able
1         4  0.980666     academy
3         1  0.972655    activity
3         3  0.014964    activity
4         3  0.992019  affordable
...     ...       ...         ...
295       2  0.372113        year
295       3  0.208383        year
295       4  0.173653        year
295       5  0.004962        year
296       5  0.985203        zoom

[347 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4, 5])

### Midwest Positive Tweets

In [16]:
# model topics in Positive Midwest Region Tweets
mw_pos_tm_tuple = perform_topic_modeling(midwest_tweets_by_sentiment[0], 3, 30)
pyLDAvis.sklearn.prepare(mw_pos_tm_tuple[0], mw_pos_tm_tuple[1], mw_pos_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.018876 -0.199923       1        1  42.075174
2     -0.192140  0.114002       2        1  29.361542
1      0.211015  0.085921       3        1  28.563285, topic_info=         Term         Freq        Total Category  logprob  loglift
272  teaching   614.000000   614.000000  Default  30.0000  30.0000
48     course   609.000000   609.000000  Default  29.0000  29.0000
182    online  1933.000000  1933.000000  Default  28.0000  28.0000
60   distance   362.000000   362.000000  Default  27.0000  27.0000
235    school   672.000000   672.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
239  semester   187.679473   399.639342   Topic3  -3.9402   0.4972
310      week   115.296121   221.489943   Topic3  -4.4274   0.6002
182    online   207.258572  1933.330492   Topic3  -3.8409  -0.9800
261   student   115.713394   738.922745   Topic3  -4.4238  -0.6010
300   virtual    92.023464   237.117419   Topic3  -4.6529   0.3065

[184 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         1  0.980878     academy
1         2  0.026510     academy
4         3  0.991563  affordable
7         2  0.992084     amazing
11        2  0.992328  assignment
...     ...       ...         ...
318       2  0.955957     writing
318       3  0.044463     writing
319       1  0.816601        year
319       2  0.091434        year
319       3  0.094587        year

[276 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

### Midwest Negative Tweets

In [17]:
# model topics in Negative Midwest Region Tweets
mw_neg_tm_tuple = perform_topic_modeling(midwest_tweets_by_sentiment[1], 3, 30)
pyLDAvis.sklearn.prepare(mw_neg_tm_tuple[0], mw_neg_tm_tuple[1], mw_neg_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.106794 -0.130883       1        1  45.566268
2     -0.158425  0.114932       2        1  32.244840
0      0.265219  0.015951       3        1  22.188892, topic_info=             Term        Freq       Total Category  logprob  loglift
269        school  133.000000  133.000000  Default  30.0000  30.0000
297       student  100.000000  100.000000  Default  29.0000  29.0000
212  online class   35.000000   35.000000  Default  28.0000  28.0000
41          class   94.000000   94.000000  Default  27.0000  27.0000
225         paper   27.000000   27.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
181     marketing   12.258118   13.489781   Topic3  -4.5074   1.4098
184          math   16.581360   22.350257   Topic3  -4.2053   1.2070
41          class   39.988378   94.589058   Topic3  -3.3250   0.6446
52         course   22.607774   58.806576   Topic3  -3.8953   0.5496
211        online   39.131988  213.867516   Topic3  -3.3467  -0.1928

[183 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
3         3  0.987102           algebra
4         3  0.960829  algebra calculus
7         1  0.800132              also
7         2  0.160026              also
7         3  0.053342              also
...     ...       ...               ...
347       2  0.932509         work home
353       1  0.113511             wrong
353       2  0.908090             wrong
356       1  0.526147              year
356       2  0.447225              year

[244 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

### Midwest Neutral Tweets


In [18]:
# model topics in Neutral Midwest Region Tweets
mw_neu_tm_tuple = perform_topic_modeling(midwest_tweets_by_sentiment[2], 3, 30)
pyLDAvis.sklearn.prepare(mw_neu_tm_tuple[0], mw_neu_tm_tuple[1], mw_neu_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.127611  0.164080       1        1  41.888681
1     -0.154215 -0.154070       2        1  38.120922
2      0.281826 -0.010011       3        1  19.990397, topic_info=                        Term        Freq       Total Category  logprob  \
83                 elearning   71.000000   71.000000  Default  30.0000   
35                     class  134.000000  134.000000  Default  29.0000   
139                 learning  207.000000  207.000000  Default  28.0000   
222                 semester   85.000000   85.000000  Default  27.0000   
216                   school  167.000000  167.000000  Default  26.0000   
..                       ...         ...         ...      ...      ...   
84   elearning earlylearning   16.247984   16.866045   Topic3  -4.2956   
275               via edchat   10.656394   11.272436   Topic3  -4.7174   
83                 elearning   64.257886   71.404997   Topic3  -2.9207   
274                      via   33.662584   42.697801   Topic3  -3.5672   
49                     covid   29.053715   50.353150   Topic3  -3.7144   

     loglift  
83   30.0000  
35   29.0000  
139  28.0000  
222  27.0000  
216  26.0000  
..       ...  
84    1.5726  
275   1.5537  
83    1.5045  
274   1.3722  
49    1.0600  

[168 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
6         1  0.936366          another
8         3  0.955180           apchat
9         3  0.955189  apchat suptchat
13        1  0.056345           august
13        2  0.901519           august
...     ...       ...              ...
279       3  0.041537  virtuallearning
290       2  0.961752             week
295       1  0.916494            would
297       1  0.019772             year
297       2  0.988596             year

[212 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

## Northeast Region Topics

In [19]:
# model topics in all Northeast Region Tweets
ne_tm_tuple = perform_topic_modeling(northeast_tweets, 5, 30)
pyLDAvis.sklearn.prepare(ne_tm_tuple[0], ne_tm_tuple[1], ne_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.045212 -0.352505       1        1  27.056006
0     -0.103687  0.111328       2        1  24.310358
1     -0.075216  0.011219       3        1  18.167757
4     -0.154588  0.169957       4        1  18.002915
2      0.378703  0.060002       5        1  12.462964, topic_info=         Term         Freq        Total Category  logprob  loglift
173    online  3041.000000  3041.000000  Default  30.0000  30.0000
44     course   984.000000   984.000000  Default  29.0000  29.0000
33      class   918.000000   918.000000  Default  28.0000  28.0000
140  learning  1344.000000  1344.000000  Default  27.0000  27.0000
229  semester   639.000000   639.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
97        get   144.312286   307.607296   Topic5  -3.7555   1.3256
166      need   118.598976   330.100226   Topic5  -3.9517   1.0588
282         u   123.407220   408.855423   Topic5  -3.9120   0.8845
24   business    76.221200   117.610945   Topic5  -4.3938   1.6487
301      work    71.953238   300.592889   Topic5  -4.4514   0.6527

[252 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         4  0.988331       access
2         4  0.995561   affordable
3         5  0.983467      algebra
4         2  0.996450         also
6         1  0.992133  application
...     ...       ...          ...
303       1  0.114957     workshop
303       3  0.881336     workshop
305       5  0.991927        would
306       3  0.998746         year
307       1  0.994169         zoom

[330 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 5, 3])

### Northeast Positive Tweets

In [25]:
ne_pos_tm_tuple = perform_topic_modeling(northeast_tweets_by_sentiment[0], 3, 30)
pyLDAvis.sklearn.prepare(ne_pos_tm_tuple[0], ne_pos_tm_tuple[1], ne_pos_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.030459 -0.197816       1        1  39.596179
1     -0.243628  0.079123       2        1  32.479548
0      0.213169  0.118694       3        1  27.924273, topic_info=                 Term        Freq        Total Category  logprob  loglift
48             course  875.000000   875.000000  Default  30.0000  30.0000
35              class  726.000000   726.000000  Default  29.0000  29.0000
287          teaching  787.000000   787.000000  Default  28.0000  28.0000
100              free  384.000000   384.000000  Default  27.0000  27.0000
171              make  328.000000   328.000000  Default  26.0000  26.0000
..                ...         ...          ...      ...      ...      ...
195            online  260.631915  2467.372653   Topic3  -3.7913  -0.9721
275           student  193.396081   884.464789   Topic3  -4.0897  -0.2446
65   distancelearning  135.702453   272.689201   Topic3  -4.4440   0.5778
250          semester  149.846919   453.609247   Topic3  -4.3448   0.1681
286           teacher  134.522540   368.551046   Topic3  -4.4527   0.2678

[186 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3         3  0.991756   affordable
8         1  0.994655  application
10        2  0.986151          art
11        1  0.987688      article
13        2  0.991574          aug
...     ...       ...          ...
334       1  0.991461        would
335       2  0.995512      writing
336       1  0.148282         year
336       2  0.003902         year
336       3  0.846771         year

[252 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

### Northeast Negative Tweets


In [24]:
ne_neg_tm_tuple = perform_topic_modeling(northeast_tweets_by_sentiment[1], 3, 30)
pyLDAvis.sklearn.prepare(ne_neg_tm_tuple[0], ne_neg_tm_tuple[1], ne_neg_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.162595  0.130519       1        1  59.509299
1      0.265095  0.021337       2        1  31.269063
2     -0.102500 -0.151856       3        1   9.221638, topic_info=              Term        Freq       Total Category  logprob  loglift
272            pay  147.000000  147.000000  Default  30.0000  30.0000
113          essay   55.000000   55.000000  Default  29.0000  29.0000
252   online class   64.000000   64.000000  Default  28.0000  28.0000
334         school  136.000000  136.000000  Default  27.0000  27.0000
265          paper   48.000000   48.000000  Default  26.0000  26.0000
..             ...         ...         ...      ...      ...      ...
337        science    5.583754   13.254658   Topic3  -4.6834   1.5191
126  fall semester    6.201365   27.715307   Topic3  -4.5785   0.8864
435           want    5.816600   19.911598   Topic3  -4.6426   1.1530
186            kid    5.764467   64.674483   Topic3  -4.6516  -0.0340
146             go    5.555980   34.849527   Topic3  -4.6884   0.5475

[156 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
6         1  0.389067               age
6         3  0.583600               age
8         2  0.969113           algebra
9         2  0.926730  algebra calculus
16        2  0.990984        assignment
...     ...       ...               ...
453       1  0.949097             would
454       2  0.967627             write
455       2  0.919611         write pay
458       1  0.182105              zoom
458       2  0.773947              zoom

[213 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

### Northeast Neutral Tweets

In [26]:
ne_neu_tm_tuple = perform_topic_modeling(northeast_tweets_by_sentiment[2], 3, 30)
pyLDAvis.sklearn.prepare(ne_neu_tm_tuple[0], ne_neu_tm_tuple[1], ne_neu_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.123496  0.168723       1        1  44.115713
1     -0.129630 -0.166019       2        1  35.168709
2      0.253126 -0.002704       3        1  20.715579, topic_info=          Term        Freq       Total Category  logprob  loglift
182   register   48.000000   48.000000  Default  30.0000  30.0000
49      course   94.000000   94.000000  Default  29.0000  29.0000
36       class  121.000000  121.000000  Default  28.0000  28.0000
12      august   41.000000   41.000000  Default  27.0000  27.0000
215    student  107.000000  107.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
237    tuesday   11.295009   14.768041   Topic3  -4.6102   1.3062
51       covid   18.387041   44.588957   Topic3  -4.1229   0.6884
18       begin   10.827283   16.076964   Topic3  -4.6525   1.1790
39   classroom   12.090075   28.350998   Topic3  -4.5422   0.7220
223   teaching   10.833551   98.818503   Topic3  -4.6519  -0.6364

[177 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2         3  0.972660        amid
3         3  0.964399  amid covid
4         1  0.065122   announced
4         2  0.911705   announced
7         3  0.903070         app
...     ...       ...         ...
258       1  0.820325     webinar
258       3  0.164065     webinar
261       2  0.934371        week
261       3  0.038932        week
265       1  0.980601        year

[235 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

## Southeast Region Topics

In [20]:
# model topics in all Southeast Region Tweets
se_tm_tuple = perform_topic_modeling(southeast_tweets, 5, 30)
pyLDAvis.sklearn.prepare(se_tm_tuple[0], se_tm_tuple[1], se_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.080724  0.329841       1        1  29.077159
1     -0.031151 -0.039656       2        1  22.255465
2     -0.210578 -0.065666       3        1  18.441598
3     -0.037970 -0.232145       4        1  18.067219
4      0.360423  0.007626       5        1  12.158559, topic_info=                Term         Freq        Total Category  logprob  loglift
168           online  2748.000000  2748.000000  Default  30.0000  30.0000
34             class   975.000000   975.000000  Default  29.0000  29.0000
217           school   994.000000   994.000000  Default  28.0000  28.0000
244          student  1108.000000  1108.000000  Default  27.0000  27.0000
52          distance   512.000000   512.000000  Default  26.0000  26.0000
..               ...          ...          ...      ...      ...      ...
130         learning   182.806204  1340.264706   Topic5  -3.4341   0.1149
45             covid    85.425149   196.315046   Topic5  -4.1949   1.2751
282  virtuallearning   113.638602   449.121116   Topic5  -3.9095   0.7329
237            start    91.823252   262.692784   Topic5  -4.1227   1.0560
253          teacher    89.394136   571.927999   Topic5  -4.1495   0.2512

[257 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2         1  0.210123    activity
2         4  0.780459    activity
3         1  0.982790  affordable
5         3  0.336409        also
5         4  0.659086        also
...     ...       ...         ...
295       3  0.031873       world
296       4  0.995555       would
297       2  0.170541        year
297       3  0.275669        year
297       5  0.553674        year

[363 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

### Southeast Positive Tweets

In [27]:
se_pos_tm_tuple = perform_topic_modeling(southeast_tweets_by_sentiment[0], 3, 30)
pyLDAvis.sklearn.prepare(se_pos_tm_tuple[0], se_pos_tm_tuple[1], se_pos_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.073733 -0.219542       1        1  38.422437
2     -0.239614  0.049890       2        1  37.918528
1      0.165881  0.169652       3        1  23.659035, topic_info=         Term         Freq        Total Category  logprob  loglift
178    online  2140.000000  2140.000000  Default  30.0000  30.0000
36      class   689.000000   689.000000  Default  29.0000  29.0000
137  learning  1021.000000  1021.000000  Default  28.0000  28.0000
232  semester   482.000000   482.000000  Default  27.0000  27.0000
265   teacher   443.000000   443.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
21       best    90.860993   181.496007   Topic3  -4.6025   0.7495
134      know    83.886641   154.509024   Topic3  -4.6824   0.8306
266  teaching   104.743595   765.563835   Topic3  -4.4603  -0.5477
137  learning   103.946641  1021.283122   Topic3  -4.4680  -0.8435
56   distance    86.377748   378.156884   Topic3  -4.6531  -0.0352

[193 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3         2  0.996303  affordable
7         1  0.984388     amazing
10        2  0.983723       apply
14        3  0.996050      august
15        2  0.009163   available
...     ...       ...         ...
305       3  0.215719        week
312       1  0.815829        year
312       2  0.180626        year
312       3  0.003010        year
314       3  0.986884        zoom

[294 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

### Southeast Negative Tweets

In [28]:
se_neg_tm_tuple = perform_topic_modeling(southeast_tweets_by_sentiment[1], 3, 30)
pyLDAvis.sklearn.prepare(se_neg_tm_tuple[0], se_neg_tm_tuple[1], se_neg_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.062826  0.138649       1        1  44.747315
2     -0.136056 -0.108335       2        1  38.743100
0      0.198882 -0.030314       3        1  16.509585, topic_info=             Term        Freq       Total Category  logprob  loglift
298        school  119.000000  119.000000  Default  30.0000  30.0000
346      teaching  103.000000  103.000000  Default  29.0000  29.0000
255           pay   31.000000   31.000000  Default  28.0000  28.0000
235        online  289.000000  289.000000  Default  27.0000  27.0000
305      semester   83.000000   83.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
235        online   48.237175  289.465848   Topic3  -2.9162   0.0093
236  online class   15.350222   48.737460   Topic3  -4.0612   0.6459
42          class   24.065703  147.940272   Topic3  -3.6116  -0.0148
59         course   11.340104   51.075159   Topic3  -4.3640   0.2963
334       student    8.808333   74.739940   Topic3  -4.6167  -0.3371

[191 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
6         3  0.873567   algebra calculus
12        3  0.976277         assignment
13        3  0.911075  assignment online
17        2  0.903823        back school
18        1  0.263797                bad
...     ...       ...                ...
393       1  0.479562               year
393       2  0.511533               year
396       1  0.905459               zoom
396       3  0.056591               zoom
397       1  0.887682         zoom class

[251 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

### Southeast Neutral Tweets

In [29]:
se_neu_tm_tuple = perform_topic_modeling(southeast_tweets_by_sentiment[2], 3, 30)
pyLDAvis.sklearn.prepare(se_neu_tm_tuple[0], se_neu_tm_tuple[1], se_neu_tm_tuple[2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.147399  0.190378       1        1  51.067537
1     -0.146405 -0.190809       2        1  28.730571
0      0.293804  0.000430       3        1  20.201891, topic_info=                 Term        Freq       Total Category  logprob  loglift
246   virtuallearning  109.000000  109.000000  Default  30.0000  30.0000
149            online  333.000000  333.000000  Default  29.0000  29.0000
197            school  163.000000  163.000000  Default  28.0000  28.0000
227          training   57.000000   57.000000  Default  27.0000  27.0000
69          elearning   53.000000   53.000000  Default  26.0000  26.0000
..                ...         ...         ...      ...      ...      ...
62   distancelearning   24.788730   44.857890   Topic3  -3.8513   1.0063
28              child   14.533966   24.206605   Topic3  -4.3852   1.0893
240               via   17.667151   59.858053   Topic3  -4.1900   0.3791
31          classroom   11.989629   20.601701   Topic3  -4.5777   1.0581
124          learning   17.969433  175.385995   Topic3  -4.1731  -0.6789

[177 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
8         2  0.954394       audience
8         3  0.095439       audience
9         2  0.888552  audience grow
11        1  0.777792         august
11        2  0.222226         august
...     ...       ...            ...
257       1  0.982395           work
259       1  0.076921          world
259       3  0.923047          world
260       2  0.902252          would
261       2  0.987985           year

[212 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])